In [2]:
from tqdm import tqdm
from IPython import display

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset

from sklearn.metrics import accuracy_score

from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq
from transformers.models.t5.modeling_t5 import T5LayerFF

from datasets import load_dataset
import gc

In [3]:
# define configuration
SMALL_MODEL_NAME = "t5-small"
BASE_MODEL_NAME = "t5-base"
LARGE_MODEL_NAME = "t5-large"

BATCH_SIZE = 8
LEARNING_RATE = 1e-4
EPOCHS = 20
BOTTLENECK_SIZE = 8

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
# load the model and tokenizer
tokenizer = T5Tokenizer.from_pretrained(SMALL_MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(SMALL_MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [5]:
# load MedMCQA
opt_idx2str = {
    0: "A",
    1: "B",
    2: "C",
    3: "D",
}
dataset = load_dataset("openlifescienceai/medmcqa")
dataset

Generating train split:   0%|          | 0/182822 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6150 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4183 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 182822
    })
    test: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 6150
    })
    validation: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 4183
    })
})

In [6]:
def filter_none(example):
    return example["exp"] is not None


dataset["validation"] = dataset["validation"].filter(filter_none)
dataset["train"] = dataset["train"].filter(filter_none)

Filter:   0%|          | 0/4183 [00:00<?, ? examples/s]

Filter:   0%|          | 0/182822 [00:00<?, ? examples/s]

In [7]:
# clean the dataset for our task
def format_example_training(row):
    input_text = f"Question: {row['question']}\n\nOptions:\nA: {row['opa']}\nB: {row['opb']}\nC: {row['opc']}\nD: {row['opd']}\n\nExplanation: {row['exp']}\n\nAnswer:"
    target_text = f"Answer: {opt_idx2str[row['cop']]}"
    return {"input_text": input_text, "target_text": target_text}


def format_example_validation(row):
    input_text = f"Question: {row['question']}\n\nOptions:\nA: {row['opa']}\nB: {row['opb']}\nC: {row['opc']}\nD: {row['opd']}\n\nExplanation: {row['exp']}\n\nAnswer:"
    target_text = f"Answer: {opt_idx2str[row['cop']]}"
    return {"input_text": input_text, "target_text": target_text}


dataset["train"] = dataset["train"].map(
    format_example_training, remove_columns=dataset["train"].column_names
)
dataset["validation"] = dataset["validation"].map(
    format_example_validation, remove_columns=dataset["validation"].column_names
)

Map:   0%|          | 0/160869 [00:00<?, ? examples/s]

Map:   0%|          | 0/2206 [00:00<?, ? examples/s]

In [8]:
# use tokenzier and prepare the data to feed the model
def map_function(row):
    input_info = tokenizer(row["input_text"], truncation=True, max_length=1024)
    output_info = tokenizer(row["target_text"])
    return {**input_info, "labels": output_info.input_ids}


dataset["train"] = dataset["train"].map(map_function, batched=True)
dataset["train"].set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"]
)

dataset["validation"] = dataset["validation"].map(map_function, batched=True)
dataset["validation"].set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"]
)

Map:   0%|          | 0/160869 [00:00<?, ? examples/s]

Map:   0%|          | 0/2206 [00:00<?, ? examples/s]

In [9]:
col_fn = DataCollatorForSeq2Seq(tokenizer, return_tensors="pt", padding="longest")

indices = list(range(0, 14000))
train_dataset = Subset(dataset["train"], indices)
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, collate_fn=col_fn, shuffle=True
)

indices = list(range(15000, 17000))
test_dataset = Subset(dataset["train"], indices)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=col_fn)

val_loader = DataLoader(dataset["validation"], batch_size=BATCH_SIZE, collate_fn=col_fn)

In [10]:
torch.cuda.empty_cache()
gc.collect()

62

In [11]:
def train_loop(model, loader, optimizer):
    model.train()

    batch_losses = []

    for row in tqdm(loader):
        row = row.to(model.device)

        optimizer.zero_grad()
        out = model(**row)
        loss = out.loss
        batch_loss_value = loss.item()
        loss.backward()
        optimizer.step()

        batch_losses.append(batch_loss_value)
    loss_value = np.mean(batch_losses)
    return {"train_loss": loss_value}


def predict(model, row):
    return model.generate(
        input_ids=row.input_ids, attention_mask=row.attention_mask, max_length=5
    )


def tokenizer_ids_to_label(all_input_ids):
    vocab_size = tokenizer.vocab_size

    filtered_input_ids = [
        [token_id for token_id in seq if 0 <= token_id < vocab_size]
        for seq in all_input_ids
    ]

    return tokenizer.batch_decode(filtered_input_ids, skip_special_tokens=True)


def valid_loop(model, loader, compute_metrics):
    model.eval()

    all_true = []
    all_pred = []

    with torch.no_grad():
        for row in tqdm(loader):
            row = row.to(model.device)
            pred = predict(model, row)

            all_true += row.labels.detach().cpu().tolist()
            all_pred += pred.detach().cpu().tolist()

    all_true = tokenizer_ids_to_label(all_true)
    all_pred = tokenizer_ids_to_label(all_pred)

    return {"valid_acc": compute_metrics(y_true=all_true, y_pred=all_pred)}

In [12]:
# adapter layer
class AdapterLayer(nn.Module):
    def __init__(self, emb_dim: int, bottleneck_size: int):

        super().__init__()

        self.sharif_llm_adapter = nn.Sequential(
            nn.Linear(emb_dim, bottleneck_size),
            nn.ReLU(),
            nn.Linear(bottleneck_size, emb_dim),
        )

    def forward(self, x: torch.Tensor):
        adapter_output = self.sharif_llm_adapter(x)
        output = x + adapter_output
        return output


class FeedForwardAdapterWrapper(nn.Module):
    def __init__(self, original_module: T5LayerFF, bottleneck_size: int):

        super().__init__()
        assert isinstance(original_module, T5LayerFF)

        self.original_module = original_module
        emb_dim = original_module.DenseReluDense.wi.in_features
        self.adapter = AdapterLayer(emb_dim, bottleneck_size)

    def forward(self, x: torch.Tensor):
        output = self.original_module(x)
        output = self.adapter(output)
        return output

In [13]:
# add adapter to the model
def mutate_model_recursive(model: nn.Module, bottleneck_size: int):
    for name, module in model.named_children():
        if isinstance(module, T5LayerFF):
            feed_forward_with_adapter = FeedForwardAdapterWrapper(
                module, bottleneck_size
            )
            setattr(model, name, feed_forward_with_adapter)
            print(f"Replaced {name} with FeedForwardAdapterWrapper layer.")
        else:
            mutate_model_recursive(module, bottleneck_size)


def mutate_model(model: nn.Module, bottleneck_size: int):
    if hasattr(model, "_mutated"):
        print("Model already contains adapter layers! \n Try reloading the model.")
        return

    mutate_model_recursive(model, bottleneck_size)

    model._mutated = True


mutate_model(model, bottleneck_size=BOTTLENECK_SIZE)

Replaced 1 with FeedForwardAdapterWrapper layer.
Replaced 1 with FeedForwardAdapterWrapper layer.
Replaced 1 with FeedForwardAdapterWrapper layer.
Replaced 1 with FeedForwardAdapterWrapper layer.
Replaced 1 with FeedForwardAdapterWrapper layer.
Replaced 1 with FeedForwardAdapterWrapper layer.
Replaced 2 with FeedForwardAdapterWrapper layer.
Replaced 2 with FeedForwardAdapterWrapper layer.
Replaced 2 with FeedForwardAdapterWrapper layer.
Replaced 2 with FeedForwardAdapterWrapper layer.
Replaced 2 with FeedForwardAdapterWrapper layer.
Replaced 2 with FeedForwardAdapterWrapper layer.


In [14]:
# freeze non adapter parameters
def freeze_non_adapter(model, peft_key):
    print("Non freezed weights:")
    total_params = 0
    for param_name, weights in model.named_parameters():
        weights.requires_grad = peft_key in param_name
        if weights.requires_grad:
            print(param_name)
            total_params += weights.numel()
    print(f"Total number of parameters should be update: {total_params}")


freeze_non_adapter(model, peft_key="sharif_llm")

Non freezed weights:
encoder.block.0.layer.1.adapter.sharif_llm_adapter.0.weight
encoder.block.0.layer.1.adapter.sharif_llm_adapter.0.bias
encoder.block.0.layer.1.adapter.sharif_llm_adapter.2.weight
encoder.block.0.layer.1.adapter.sharif_llm_adapter.2.bias
encoder.block.1.layer.1.adapter.sharif_llm_adapter.0.weight
encoder.block.1.layer.1.adapter.sharif_llm_adapter.0.bias
encoder.block.1.layer.1.adapter.sharif_llm_adapter.2.weight
encoder.block.1.layer.1.adapter.sharif_llm_adapter.2.bias
encoder.block.2.layer.1.adapter.sharif_llm_adapter.0.weight
encoder.block.2.layer.1.adapter.sharif_llm_adapter.0.bias
encoder.block.2.layer.1.adapter.sharif_llm_adapter.2.weight
encoder.block.2.layer.1.adapter.sharif_llm_adapter.2.bias
encoder.block.3.layer.1.adapter.sharif_llm_adapter.0.weight
encoder.block.3.layer.1.adapter.sharif_llm_adapter.0.bias
encoder.block.3.layer.1.adapter.sharif_llm_adapter.2.weight
encoder.block.3.layer.1.adapter.sharif_llm_adapter.2.bias
encoder.block.4.layer.1.adapter.sha

In [15]:
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
compute_metrics = accuracy_score

In [ ]:
model.to(DEVICE)

all_results = []
for epoch in range(EPOCHS):
    epoch_results = {"epoch": epoch}

    epoch_results.update(
        train_loop(model=model, loader=train_loader, optimizer=optimizer)
    )
    epoch_results.update(
        valid_loop(model=model, loader=val_loader, compute_metrics=compute_metrics)
    )
    all_results.append(epoch_results)

    display.clear_output()
    display.display(pd.DataFrame(all_results).set_index("epoch"))

,train_loss,valid_acc
epoch,,
0,0.495406,0.685403
1,0.238696,0.755666
2,0.209013,0.760653
3,0.193738,0.766999
4,0.175897,0.785585
5,0.166662,0.791931
6,0.159725,0.790118


 93%|█████████▎| 1622/1750 [02:53<00:11, 11.24it/s]

In [ ]:
model.eval()

all_true = []
all_pred = []

with torch.no_grad():
    for row in tqdm(test_loader):
        row = row.to(model.device)
        pred = predict(model, row)

        all_true += row.labels.detach().cpu().tolist()
        all_pred += pred.detach().cpu().tolist()

all_true = tokenizer_ids_to_label(all_true)
all_pred = tokenizer_ids_to_label(all_pred)
print(compute_metrics(y_true=all_true, y_pred=all_pred))

In [ ]:
print(all_true)
print(all_pred)

In [ ]:
torch.save(model.state_dict(), "t5_adapter_weights.pth")

In [ ]:
# code to generate answer based on model
def generate_answer(row):
    input_text = f"Question: {row['question']}\n\nOptions:\nA: {row['opa']}\nB: {row['opb']}\nC: {row['opc']}\nD: {row['opd']}\n\nExplanation: {row['exp']}\n\nAnswer:"
    input_ids = tokenizer(input_text, truncation=True, max_length=1024)
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    outputs = model.generate(input_ids.to(DEVICE), max_length=5)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

In [ ]:
t = {
    "question": "Which of the following is most characteristic of diabetic neuropathy?",
    "opa": "it is usually bilateral",
    "opb": "pain is not a feature",
    "opc": "it most commonly affects the brain",
    "opd": "it spares the autonomic system",
    "cop": 0,
    # "exp": "Diabetic neuropathy usually presents as peripheral polyneuropathy, usually bilateral, including symptoms of numbness, paresthesia, severe hyperesthesia, and pain. Impairment of proprioceptive fibers can lead to gait abnormalities and Charcot's joints. Mononeuropathy is less common and is often spontaneously reversible. Common syndromes include wrist or foot drop and third, fourth, or sixth cranial nerve palsies. Autonomic neuropathy may cause gastroesophageal dysfunction, bladder dysfunction, and orthostatic hypotension.",
}

In [ ]:
answer = generate_answer(t)
print(f"Model's Answer: {answer}")
print(f"True Answer: {opt_idx2str[t["cop"]]}")